In [339]:
import os
import copy
import re
import pandas as pd
import numpy as np
import csv
import argparse


# get current(script) working directory
script_path = os.getcwd()
print(script_path)

# get project folder directory
project_path = os.path.dirname(script_path)
print(project_path)

# get strain folder directory
strain_path = os.path.join(project_path,"data/quality_variants")
print(strain_path)

# get reference chromosom folder directory
ref_path = os.path.join(project_path,"data/reference")
print(ref_path)

# get a list of chromosome files' name
chr_files = [file for file in os.listdir(ref_path)]
print(chr_files)

chrfiles_dict = {}
for name in chr_files:
    chrfiles_dict[name[10]] = name
print(chrfiles_dict)

# get a list of strain files' name
strain_files = [file for file in os.listdir(strain_path)]
print(strain_files)


# # use an Argument Parser object to handle script arguments
# parser = argparse.ArgumentParser()
# parser.add_argument("-chr_index", type=str, help="chromosom 1-5, C or M")
# parser.add_argument("-start", type=int, help="starting base position")
# parser.add_argument("-end", type=int, help="ending base position")
# args = parser.parse_args()

def mutant_alignment(chr_index, start, end):
    """Input
    start: the index of start base position in a chrmosome(starting at 1)
    end: the index of end base position in a chromosome
    chr_index: the index of specific chrmosome(in 1-5, C or M)
    Examples:
    mutant_alignment("1", 1, 10)
    """

    
    # convert int to string
    if type(chr_index) == int:
        chr_index = str(chr_index)
    
    assert start > 0 and type(start) == int,"error message: start index must be a positive integer!"
    assert end > 0 and type(start) == int,"error message: end index must be a positive integer \
                                                larger than start index and within chromosome length!"

    # Use start and end position to find which lines should be read in
    # Since each file will have 80 bases on each line, so we get the quotient of start/80 as start line index
    # and use the quotient of end/80 as the end line index
    line_start = int(start/80)
    line_end = int(end/80) + 1
    
    start_new = start - line_start*80
    end_new = end - line_start*80
    
    print(line_start, line_end, start_new, end_new)
    # Get the path for each chromosome file
    chrfile_name = chrfiles_dict[chr_index]
    chrfile = os.path.join(ref_path,chrfile_name)
    print(chrfile)
    
    chr_seq = ""
    with open(chrfile) as chromosome:
        next(chromosome) # Skip the first line since it does not contain bases
        lines = [line.strip('\n') for line in chromosome][line_start:line_end]
        
    chr_seq = "".join(lines)
    print(chr_seq)
    
    # start-1 and end-1 to get right index in python, since python starts from 0
    # narrow to the sequence we need
    chr_seq = chr_seq[start_new-1:end_new]
    # change string to list for future modification
    chr_seq_list = list(chr_seq)
    print(chr_seq_list)
        
    output = []
    chromosome = 'chr' + chr_index
    print(chromosome)
    
    # read in strain files and extract useful information
    for strainfile_name in strain_files[0:10]:
        strainfile = os.path.join(strain_path, strainfile_name)
        strain =  pd.read_csv(strainfile, header = None, names = ["strain", "chr", "pos", "ref_base", "sub_base", "qua", "non", "con", "avg"], sep = '\t')
        
        # limit data within specific range (specified chrmosome and position between start and end)
        strain = strain[(strain.chr == chromosome) & (strain.pos >= start) & (strain.pos <= end)]
        
        # get strain name
        strain_name = re.split('quality_variant_|.txt', strainfile_name)[1]
        
        #begin substitution
        if strain.empty == False:
            chr_seq_list_mutant = copy.deepcopy(chr_seq_list)
            for i in range(len(strain)):
                position = strain.iloc[i,2] - 1 - line_start*80 - start_new
                chr_seq_list_mutant[position] = strain.iloc[i,4]
            output.append([strain_name, "".join(chr_seq_list_mutant)])
        else:
            output.append([strain_name, "".join(chr_seq_list)])
            
    total_bases = end - start + 1
    total_strains = len(strain_files)
    output.insert(0, [total_strains, total_bases])
    
    if len(str(start)) == 1:
        start_string = "00000" + str(start)
    elif len(str(start)) == 2:
        start_string = "0000" + str(start)
    elif len(str(start)) == 3:
        start_string = "000" + str(start)
    elif len(str(start)) == 4:
        start_string = "00" + str(start)
    elif len(str(start)) == 5:
        start_string = "0" + str(start)
    elif len(str(start)) == 6:
        start_string = str(start)
        
    if len(str(end)) == 1:
        end_string = "00000" + str(end)
    elif len(str(end)) == 2:
        end_string = "0000" + str(end)
    elif len(str(end)) == 3:
        end_string = "000" + str(end)
    elif len(str(end)) == 4:
        end_string = "00" + str(end)
    elif len(str(end)) == 5:
        end_string = "0" + str(end)
    elif len(str(end)) == 6:
        end_string = str(end)
    
    output_file = chromosome + "_" + start_string + "_to_" + end_string + ".phy"
    
    with open(output_file, 'w', newline='') as f:
        csv_writer = csv.writer(f,delimiter =' ')
        csv_writer.writerows(output)
    
    return output

E:\Study\UWM\STAT679\project-team-12\scripts
E:\Study\UWM\STAT679\project-team-12
E:\Study\UWM\STAT679\project-team-12\data/quality_variants
E:\Study\UWM\STAT679\project-team-12\data/reference
['TAIR10_chr1.fas', 'TAIR10_chr2.fas', 'TAIR10_chr3.fas', 'TAIR10_chr4.fas', 'TAIR10_chr5.fas', 'TAIR10_chrC.fas', 'TAIR10_chrM.fas']
{'1': 'TAIR10_chr1.fas', '2': 'TAIR10_chr2.fas', '3': 'TAIR10_chr3.fas', '4': 'TAIR10_chr4.fas', '5': 'TAIR10_chr5.fas', 'C': 'TAIR10_chrC.fas', 'M': 'TAIR10_chrM.fas'}
['quality_variant_Aa_0.txt', 'quality_variant_Abd_0.txt', 'quality_variant_Ag_0.txt', 'quality_variant_Ak_1.txt', 'quality_variant_Alst_1.txt', 'quality_variant_Altai_5.txt', 'quality_variant_Amel_1.txt', 'quality_variant_Ang_0.txt', 'quality_variant_Anholt_1.txt', 'quality_variant_Ann_1.txt', 'quality_variant_Anz_0.txt', 'quality_variant_An_1.txt', 'quality_variant_Appt_1.txt', 'quality_variant_Baa_1.txt', 'quality_variant_Ba_1.txt', 'quality_variant_Bch_1.txt', 'quality_variant_Bd_0.txt', 'quality

In [340]:
mutant_alignment(1, 1, 5)

0 1 1 5
E:\Study\UWM\STAT679\project-team-12\data/reference\TAIR10_chr1.fas
CCCTAAACCCTAAACCCTAAACCCTAAACCTCTGAATCCTTAATCCCTAAATCCCTAAATCTTTAAATCCTACATCCAT
['C', 'C', 'C', 'T', 'A']
chr1


[[216, 5],
 ['Aa_0', 'CCCTA'],
 ['Abd_0', 'CCCTA'],
 ['Ag_0', 'CCCTA'],
 ['Ak_1', 'CCCTA'],
 ['Alst_1', 'CCCTA'],
 ['Altai_5', 'CCCTA'],
 ['Amel_1', 'CCCTA'],
 ['Ang_0', 'CCCTA'],
 ['Anholt_1', 'CCCTA'],
 ['Ann_1', 'CCCTA']]